In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from scipy import stats


In [2]:
data = pd.read_csv("SHEL2.csv")

data.head()

,Date,Open,High,Low,Close,Returns,Y_variable,Adj Close,Volume
0,10/4/2011,1940.0,1952.5,1886.0,1928.5,0.000000,0,1051.180908,3724030
1,10/5/2011,1970.0,2018.0,1964.5,2000.5,0.037335,1,1090.426392,3449487
2,10/6/2011,2018.5,2056.0,1996.5,2056.0,0.027743,1,1120.677856,4535816
3,10/7/2011,2043.5,2089.5,2038.5,2069.0,0.006323,1,1127.764038,2429486
4,10/10/2011,2082.0,2132.5,2069.0,2120.0,0.024650,1,1155.562622,2063653


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2777 entries, 0 to 2776
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        2777 non-null   object 
 1   Open        2777 non-null   float64
 2   High        2777 non-null   float64
 3   Low         2777 non-null   float64
 4   Close       2777 non-null   float64
 5   Returns     2777 non-null   float64
 6   Y_variable  2777 non-null   int64  
 7   Adj Close   2777 non-null   float64
 8   Volume      2777 non-null   int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 195.4+ KB


In [4]:
returns_prices = data['Returns'] # dependet variable

#Open +High + Low +  Volume =  4 independent variables X 
# genfromtxt Load data from a text file, with missing values handled as specified.
data2 = np.genfromtxt("SHEL2.csv", dtype='float', delimiter=',', skip_header=1, usecols=(1,2,3,8))

X = np.array(data2)
print(X)

[[1.9400000e+03 1.9525000e+03 1.8860000e+03 3.7240300e+06]
 [1.9700000e+03 2.0180000e+03 1.9645000e+03 3.4494870e+06]
 [2.0185000e+03 2.0560000e+03 1.9965000e+03 4.5358160e+06]
 ...
 [2.2615000e+03 2.2715000e+03 2.2195000e+03 1.7127418e+07]
 [2.2525000e+03 2.2820000e+03 2.2285000e+03 1.6698217e+07]
 [2.2550000e+03 2.3155000e+03 2.2475000e+03 1.5578724e+07]]


In [6]:
# Split the data/target into training/testing sets
X_train = np.array(X[:2222]).reshape((-1, 4)) # 2222 elementet e pare
X_test = np.array(X[2222:]).reshape((-1, 4)) #pas  2222 index

Y_train = returns_prices[:2222] 
Y_test = returns_prices[2222:] 


regr = linear_model.LinearRegression()


regr.fit(X_train, Y_train)

LinearRegression()

In [7]:
#--------------------------------------------
params = np.append(regr.intercept_,regr.coef_)
params



array([-1.29973583e-02, -5.42403511e-04,  5.31511075e-04,  1.37494734e-05,
       -4.21825650e-10])

In [8]:
predictions = regr.predict(X_train)

In [9]:
from sklearn.metrics import r2_score
r2 = r2_score(Y_train, predictions)
r2

0.3109411553107516

In [10]:
newX = np.append(np.ones((len(X_train),1)), X_train, axis=1)
MSE = (sum((Y_train-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX[0])))) for i in ts_b]
p_values

[2.9545899282723553e-08, 0.0, 0.0, 7.165776692952264e-05, 0.0]

In [11]:
sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)
params

array([-0.013 , -0.0005,  0.0005,  0.    , -0.    ])

In [12]:
myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilities"] = [params,sd_b,ts_b,p_values]
print(myDF3)

   Coefficients  Standard Errors  t values  Probabilities
0       -0.0130            0.002    -5.564            0.0
1       -0.0005            0.000   -31.507            0.0
2        0.0005            0.000    30.932            0.0
3        0.0000            0.000     3.978            0.0
4       -0.0000            0.000    -8.481            0.0


In [13]:
import statsmodels.api as sm
X = sm.add_constant(X)

# fit linear model
model = sm.OLS(returns_prices,X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Returns   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     281.0
Date:                Mon, 25 Nov 2024   Prob (F-statistic):          4.88e-203
Time:                        19:59:59   Log-Likelihood:                 7740.9
No. Observations:                2777   AIC:                        -1.547e+04
Df Residuals:                    2772   BIC:                        -1.544e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0087      0.002     -5.049      0.0